***Классификация пород кошек и собак по лицу***

Для начала загрузим упрощенный датасет - [Oxford-IIIT Dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/)

In [1]:
#lib import
from keras import utils
from keras import layers
from keras.models import Sequential
from keras.metrics import SparseTopKCategoricalAccuracy
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
import time
from pathlib import Path

In [2]:
#dataset loading
url = 'http://www.soshnikov.com/permanent/data/petfaces.tar.gz'
data = Path(utils.get_file('petfaces', origin=url, untar=True))

24483412/24483412 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [3]:
#dividing to test and example 
img_size = 256, 256

train_set = utils.image_dataset_from_directory (
    data, 
    validation_split=0.2, 
    subset='training',
    seed=3000,
    image_size=img_size,
    batch_size=32)

test_set = utils.image_dataset_from_directory (
    data, 
    validation_split=0.2, 
    subset='validation',
    seed=3000, 
    image_size=img_size,
    batch_size=32)

Found 3211 files belonging to 35 classes.
Using 2569 files for training.
Found 3211 files belonging to 35 classes.
Using 642 files for validation.


Далее разделим полученный датасет на классы: кошки и собаки (cats/dogs)

In [4]:
class_names = train_set.class_names

class_cats = []
class_dogs = []

for _ in range(len(class_names)):
    if class_names[_].startswith('cat'):
        class_cats.append(_)
    if class_names[_].startswith('dog'):
        class_dogs.append(_)

train_set = train_set.cache().shuffle(3000 // 2).prefetch(buffer_size=tf.data.AUTOTUNE)
test_set = test_set.cache().prefetch(buffer_size=tf.data.AUTOTUNE)